In [ ]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torchvision.utils as vutils
from torch.utils.data import DataLoader, Dataset
from pytorch_fid import fid_score
from PIL import Image

# Set parameters
IMG_SIZE = 64

BATCH_SIZE = 64
LATENT_DIM = 100
EPOCHS = 20
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Preprocess dataset
transform = transforms.Compose([
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.ToTensor(),
    transforms.Normalize([0.5], [0.5])
])

# Custom Dataset Class
class ImageDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.image_files = [os.path.join(root_dir, f) for f in os.listdir(root_dir) if f.endswith('.jpg')]

    def __len__(self):
        return len(self.image_files)

    def __getitem__(self, idx):
        img_path = self.image_files[idx]
        image = Image.open(img_path).convert('RGB')
        if self.transform:
            image = self.transform(image)
        return image  # No labels, just images

# Load Dataset
dataset = ImageDataset(root_dir='images', transform=transform)
dataloader = DataLoader(dataset, batch_size=BATCH_SIZE, shuffle=True)

# Define Generator
class Generator(nn.Module):
    def __init__(self):
        super(Generator, self).__init__()
        self.main = nn.Sequential(
            nn.ConvTranspose2d(LATENT_DIM, 512, 4, 1, 0, bias=False),
            nn.BatchNorm2d(512),
            nn.ReLU(True),
            nn.ConvTranspose2d(512, 256, 4, 2, 1, bias=False),
            nn.BatchNorm2d(256),
            nn.ReLU(True),
            nn.ConvTranspose2d(256, 128, 4, 2, 1, bias=False),
            nn.BatchNorm2d(128),
            nn.ReLU(True),
            nn.ConvTranspose2d(128, 3, 4, 2, 1, bias=False),
            nn.Tanh()
        )

    def forward(self, x):
        return self.main(x)

# Define Discriminator
class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()
        self.main = nn.Sequential(
            nn.Conv2d(3, 128, 4, 2, 1, bias=False),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(128, 256, 4, 2, 1, bias=False),
            nn.BatchNorm2d(256),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(256, 512, 4, 2, 1, bias=False),
            nn.BatchNorm2d(512),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(512, 1, 4, 1, 0, bias=False),
            nn.Sigmoid()
        )

    def forward(self, x):
        return self.main(x).view(-1, 1)  # Ensures output shape is (batch_size, 1)

# Initialize models
gen = Generator().to(device)
disc = Discriminator().to(device)
criterion = nn.BCELoss()

# Optimizers to test
optimizers = {
    'Adam': optim.Adam,
    'RMSprop': optim.RMSprop,
    'SGD': optim.SGD
}

results = {}

for opt_name, opt_class in optimizers.items():
    print(f"Training with {opt_name} optimizer...")
    optimizer_g = opt_class(gen.parameters(), lr=0.0002)
    optimizer_d = opt_class(disc.parameters(), lr=0.0002)
    
    loss_g_list, loss_d_list = [], []
    
    for epoch in range(EPOCHS):
        for i, real_images in enumerate(dataloader):
            real_images = real_images.to(device)
            batch_size = real_images.size(0)
            
            # Train Discriminator
            optimizer_d.zero_grad()
            output_real = disc(real_images)
            real_labels = torch.ones_like(output_real, device=device)
            fake_labels = torch.zeros_like(output_real, device=device)

            loss_real = criterion(output_real, real_labels)
            
            noise = torch.randn(batch_size, LATENT_DIM, 1, 1, device=device)
            fake_images = gen(noise)
            output_fake = disc(fake_images.detach())
            loss_fake = criterion(output_fake, fake_labels)
            
            loss_d = loss_real + loss_fake
            loss_d.backward()
            optimizer_d.step()
            
            # Train Generator
            optimizer_g.zero_grad()
            output_fake = disc(fake_images)
            loss_g = criterion(output_fake, real_labels)
            loss_g.backward()
            optimizer_g.step()
            
            loss_g_list.append(loss_g.item())
            loss_d_list.append(loss_d.item())
        
        print(f"Epoch {epoch+1}/{EPOCHS}: Loss_D={loss_d.item():.4f}, Loss_G={loss_g.item():.4f}")
    
    # Save loss results
    results[opt_name] = {
        'Generator Loss': loss_g_list,
        'Discriminator Loss': loss_d_list
    }

# Compute FID Score for evaluation
fid_values = {}
for opt_name in optimizers.keys():
    fake_images = gen(torch.randn(500, LATENT_DIM, 1, 1, device=device)).detach().cpu()
    fake_images_path = f'fake_images_{opt_name}/'
    os.makedirs(fake_images_path, exist_ok=True)
    for i, img in enumerate(fake_images):
        vutils.save_image(img, os.path.join(fake_images_path, f'{i}.png'), normalize=True)
    
    fid = fid_score.calculate_fid_given_paths(['images/', fake_images_path], batch_size=50, device=device, dims=192)
    fid_values[opt_name] = fid

print("FID Scores:", fid_values)


Training with Adam optimizer...


ValueError: Using a target size (torch.Size([1600, 1])) that is different to the input size (torch.Size([64, 1, 5, 5])) is deprecated. Please ensure they have the same size.